In [1]:
import csv
import random
import pandas as pd
from tqdm import tqdm
from itertools import islice
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
from datasets import load_dataset

SEED = 1337
random.seed(SEED)

INFO 02-18 16:43:38 __init__.py:190] Automatically detected platform cuda.


In [2]:
dataset = load_dataset("liamdugan/raid", "raid")["train"]
dataset

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'adv_source_id', 'source_id', 'model', 'decoding', 'repetition_penalty', 'attack', 'domain', 'title', 'prompt', 'generation'],
    num_rows: 5615820
})

In [3]:
dataset = dataset.filter(lambda x: x["model"] == "human")
df = dataset.to_pandas()
df.head()

,id,adv_source_id,source_id,model,decoding,repetition_penalty,attack,domain,title,prompt,generation
0,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,human,None,None,none,abstracts,FUTURE-AI: Guiding Principles and Consensus Re...,None,The recent advancements in artificial intellig...
1,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,human,None,None,none,abstracts,EdgeFlow: Achieving Practical Interactive Segm...,None,High-quality training data play a key role in ...
2,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,human,None,None,none,abstracts,Semi-supervised Contrastive Learning for Label...,None,The success of deep learning methods in medica...
3,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,human,None,None,none,abstracts,Combo Loss: Handling Input and Output Imbalanc...,None,Simultaneous segmentation of multiple organs f...
4,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,human,None,None,none,abstracts,Attention-Based 3D Seismic Fault Segmentation ...,None,Detection faults in seismic data is a crucial ...


In [4]:
df.isna().sum()

id                         0
adv_source_id              0
source_id                  0
model                      0
decoding              160452
repetition_penalty    160452
attack                     0
domain                     0
title                      0
prompt                160452
generation                 0
dtype: int64

In [5]:
df["title_length"] = df["title"].str.len()
df["generation_length"] = df["generation"].str.len()

In [6]:
df.sort_values("title_length", ascending=False)

,id,adv_source_id,source_id,model,decoding,repetition_penalty,attack,domain,title,prompt,generation,title_length,generation_length
113783,9aee8137-24a4-429c-ae05-8c0cc74b46da,f8e94548-1ce4-4273-8e63-5b8e9a83d77f,f8e94548-1ce4-4273-8e63-5b8e9a83d77f,human,None,None,perplexity_misspelling,reddit,I'm proud of my heritage &amp; proud to be Asi...,None,There are plenty of tourists from other countr...,308,818
122678,8b95a7f2-4e44-4f03-a78b-18c8435f1801,f8e94548-1ce4-4273-8e63-5b8e9a83d77f,f8e94548-1ce4-4273-8e63-5b8e9a83d77f,human,None,None,article_deletion,reddit,I'm proud of my heritage &amp; proud to be Asi...,None,There are plenty of tourists from other countr...,308,810
115562,afc55c50-1931-4ba3-aa6c-82fbdedcff32,f8e94548-1ce4-4273-8e63-5b8e9a83d77f,f8e94548-1ce4-4273-8e63-5b8e9a83d77f,human,None,None,paraphrase,reddit,I'm proud of my heritage &amp; proud to be Asi...,None,It’s a funny thing. There are so many tourists...,308,814
126236,d983b4d7-ad84-40f2-bb85-366feba12d07,f8e94548-1ce4-4273-8e63-5b8e9a83d77f,f8e94548-1ce4-4273-8e63-5b8e9a83d77f,human,None,None,zero_width_space,reddit,I'm proud of my heritage &amp; proud to be Asi...,None,T​h​e​r​e​ ​a​r​e​ ​p​l​e​n​t​y​ ​o​f​ ​t​o​u​...,308,1636
112004,b4c2f62f-1a10-4108-b85b-a868583f91e2,f8e94548-1ce4-4273-8e63-5b8e9a83d77f,f8e94548-1ce4-4273-8e63-5b8e9a83d77f,human,None,None,synonym,reddit,I'm proud of my heritage &amp; proud to be Asi...,None,There are plenty of people from other nations ...,308,790
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82129,abfb4f42-d4a5-416c-a872-ea4cbd9294ca,03a92e4e-9f02-48ee-aca5-9f48d92e631b,03a92e4e-9f02-48ee-aca5-9f48d92e631b,human,None,None,alternative_spelling,poetry,X,None,I just feel so down\nI only frown\nI have no s...,1,1431
69732,aab7ff73-bf2e-4539-812b-d37579cbfb8b,03a92e4e-9f02-48ee-aca5-9f48d92e631b,03a92e4e-9f02-48ee-aca5-9f48d92e631b,human,None,None,synonym,poetry,X,None,I just feel so down\nI only frown\nI have no s...,1,1444
71503,885c7c40-810c-4120-bf7a-ee6628e2d6a1,03a92e4e-9f02-48ee-aca5-9f48d92e631b,03a92e4e-9f02-48ee-aca5-9f48d92e631b,human,None,None,perplexity_misspelling,poetry,X,None,I just feel so down\nI only frown\nI have no s...,1,1431
78587,56b1d77e-69c9-4d29-9b9d-8bf03da8900d,03a92e4e-9f02-48ee-aca5-9f48d92e631b,03a92e4e-9f02-48ee-aca5-9f48d92e631b,human,None,None,homoglyph,poetry,X,None,І just fееl sо dоwn\nІ оnlу frоwn\nІ hаvе nо s...,1,1431


In [7]:
df.sort_values("generation_length", ascending=False)

,id,adv_source_id,source_id,model,decoding,repetition_penalty,attack,domain,title,prompt,generation,title_length,generation_length
83933,803e82a2-f031-4549-ac47-f7d99e2556aa,ba673ef7-a429-4bfb-9d50-8486653e5a34,ba673ef7-a429-4bfb-9d50-8486653e5a34,human,None,None,zero_width_space,poetry,The House Of Dust Complete,None,I​.​\n​T​h​e​ ​s​u​n​ ​g​o​e​s​ ​d​o​w​n​ ​i​n...,26,140088
84514,2ad772b8-bc75-4b91-88a3-9b60eace05f4,1b6c8ffe-d7c4-4c42-9a08-f6a550432f8f,1b6c8ffe-d7c4-4c42-9a08-f6a550432f8f,human,None,None,zero_width_space,poetry,Paradise Lost Book X,None,T​h​u​s​ ​t​h​e​y​ ​i​n​ ​l​o​w​l​i​e​s​t​ ​p​...,20,132608
83488,0a3f876d-929f-42fb-aaf5-bad6b11e3a4d,aa11db54-f053-438a-828e-089d576cea4b,aa11db54-f053-438a-828e-089d576cea4b,human,None,None,zero_width_space,poetry,The Interpretation Of Nature And,None,"I​.​\n​M​A​N​,​ ​b​e​i​n​g​ ​t​h​e​ ​s​e​r​v​a...",32,95110
83976,7e464827-5eb1-4a02-968d-9658b9576f04,c9e580f7-e928-43eb-bc92-e2cadd174f88,c9e580f7-e928-43eb-bc92-e2cadd174f88,human,None,None,zero_width_space,poetry,Paradise Lost Book06,None,A​l​l​ ​n​i​g​h​t​ ​t​h​e​ ​d​r​e​a​d​l​e​s​s​...,20,79060
83420,6ae0759d-3f31-46ca-9b1f-1f4d06114114,3f2999d0-c90d-419a-8d15-e5f586b31a9c,3f2999d0-c90d-419a-8d15-e5f586b31a9c,human,None,None,zero_width_space,poetry,Paradise Lost Book05,None,"N​o​w​ ​M​o​r​n​,​ ​h​e​r​ ​r​o​s​y​ ​s​t​e​p​...",20,78496
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132775,6e7d1d2c-06e5-4a77-81a5-aad904bf5213,f59280c3-2b9f-49a5-b5b4-da0f2adcd77d,f59280c3-2b9f-49a5-b5b4-da0f2adcd77d,human,None,None,paraphrase,reviews,Logan,None,No words can describe the epic majesty of this...,5,52
133344,2fe4756f-dfa2-4671-a300-9d51fbfef75b,c4929f54-a6ea-4cc9-886a-8d90d9d22fe5,c4929f54-a6ea-4cc9-886a-8d90d9d22fe5,human,None,None,paraphrase,reviews,X-Men: Apocalypse,None,"""What's happened to him?"" APOLLON:",17,34
115825,ea247e9d-3c84-4998-87bf-a8f51925ffe0,dee3ab2a-ec9f-438a-b5cb-f32bb2fd1629,dee3ab2a-ec9f-438a-b5cb-f32bb2fd1629,human,None,None,paraphrase,reddit,I found this in the files when u open it up on...,None,I'm a blind man and a sad one.,54,30
116755,db04e003-a7d8-454b-ae6a-2d74fb0deeda,253f18bf-bad3-45b0-bab7-05f178d47b1b,253f18bf-bad3-45b0-bab7-05f178d47b1b,human,None,None,paraphrase,reddit,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,None,Let us hear it out:,300,19


In [9]:
df = df[(df["title_length"] >= 10) & (df["generation_length"] >= 50)]
df

,id,adv_source_id,source_id,model,decoding,repetition_penalty,attack,domain,title,prompt,generation,title_length,generation_length
0,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,human,None,None,none,abstracts,FUTURE-AI: Guiding Principles and Consensus Re...,None,The recent advancements in artificial intellig...,125,1500
1,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,human,None,None,none,abstracts,EdgeFlow: Achieving Practical Interactive Segm...,None,High-quality training data play a key role in ...,76,1021
2,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,human,None,None,none,abstracts,Semi-supervised Contrastive Learning for Label...,None,The success of deep learning methods in medica...,83,1457
3,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,human,None,None,none,abstracts,Combo Loss: Handling Input and Output Imbalanc...,None,Simultaneous segmentation of multiple organs f...,75,1816
4,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,human,None,None,none,abstracts,Attention-Based 3D Seismic Fault Segmentation ...,None,Detection faults in seismic data is a crucial ...,79,1463
...,...,...,...,...,...,...,...,...,...,...,...,...,...
160447,9b41d56c-514c-4bef-abc9-04a13f03d80b,0f9ee3c6-6ef3-4d3d-b7a7-c36705352ef8,0f9ee3c6-6ef3-4d3d-b7a7-c36705352ef8,human,None,None,zero_width_space,wiki,August Hoch,None,A​u​g​u​s​t​ ​H​o​c​h​ ​(​2​0​ ​A​p​r​i​l​ ​1​...,11,2342
160448,d4c189b3-5824-4715-8188-29e1289ea73d,a48066a1-63ed-4969-8ff2-ee123e3b9cbf,a48066a1-63ed-4969-8ff2-ee123e3b9cbf,human,None,None,zero_width_space,wiki,Made V.I.P Tour,None,T​h​e​ ​M​a​d​e​ ​V​.​I​.​P​ ​T​o​u​r​ ​i​s​ ​...,15,2376
160449,b10e5895-70c1-498a-9d27-022c5332d88f,b1e69455-73c9-47b6-a849-da38cdef929a,b1e69455-73c9-47b6-a849-da38cdef929a,human,None,None,zero_width_space,wiki,Preferred number,None,"I​n​ ​i​n​d​u​s​t​r​i​a​l​ ​d​e​s​i​g​n​,​ ​p​...",16,2294
160450,6c774f80-4ad7-44ba-9b5e-b41ed280a04f,c9b8f9f6-8c41-45b0-9509-2466f79d0346,c9b8f9f6-8c41-45b0-9509-2466f79d0346,human,None,None,zero_width_space,wiki,Francisco Bethencourt,None,F​r​a​n​c​i​s​c​o​ ​B​e​t​h​e​n​c​o​u​r​t​ ​i​...,21,2412


In [10]:
df.duplicated(subset="generation").sum()

np.int64(12928)

In [11]:
df.drop_duplicates(subset=["generation"], inplace=True)

In [12]:
df

,id,adv_source_id,source_id,model,decoding,repetition_penalty,attack,domain,title,prompt,generation,title_length,generation_length
0,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,human,None,None,none,abstracts,FUTURE-AI: Guiding Principles and Consensus Re...,None,The recent advancements in artificial intellig...,125,1500
1,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,human,None,None,none,abstracts,EdgeFlow: Achieving Practical Interactive Segm...,None,High-quality training data play a key role in ...,76,1021
2,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,human,None,None,none,abstracts,Semi-supervised Contrastive Learning for Label...,None,The success of deep learning methods in medica...,83,1457
3,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,human,None,None,none,abstracts,Combo Loss: Handling Input and Output Imbalanc...,None,Simultaneous segmentation of multiple organs f...,75,1816
4,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,human,None,None,none,abstracts,Attention-Based 3D Seismic Fault Segmentation ...,None,Detection faults in seismic data is a crucial ...,79,1463
...,...,...,...,...,...,...,...,...,...,...,...,...,...
160447,9b41d56c-514c-4bef-abc9-04a13f03d80b,0f9ee3c6-6ef3-4d3d-b7a7-c36705352ef8,0f9ee3c6-6ef3-4d3d-b7a7-c36705352ef8,human,None,None,zero_width_space,wiki,August Hoch,None,A​u​g​u​s​t​ ​H​o​c​h​ ​(​2​0​ ​A​p​r​i​l​ ​1​...,11,2342
160448,d4c189b3-5824-4715-8188-29e1289ea73d,a48066a1-63ed-4969-8ff2-ee123e3b9cbf,a48066a1-63ed-4969-8ff2-ee123e3b9cbf,human,None,None,zero_width_space,wiki,Made V.I.P Tour,None,T​h​e​ ​M​a​d​e​ ​V​.​I​.​P​ ​T​o​u​r​ ​i​s​ ​...,15,2376
160449,b10e5895-70c1-498a-9d27-022c5332d88f,b1e69455-73c9-47b6-a849-da38cdef929a,b1e69455-73c9-47b6-a849-da38cdef929a,human,None,None,zero_width_space,wiki,Preferred number,None,"I​n​ ​i​n​d​u​s​t​r​i​a​l​ ​d​e​s​i​g​n​,​ ​p​...",16,2294
160450,6c774f80-4ad7-44ba-9b5e-b41ed280a04f,c9b8f9f6-8c41-45b0-9509-2466f79d0346,c9b8f9f6-8c41-45b0-9509-2466f79d0346,human,None,None,zero_width_space,wiki,Francisco Bethencourt,None,F​r​a​n​c​i​s​c​o​ ​B​e​t​h​e​n​c​o​u​r​t​ ​i​...,21,2412


In [13]:
def batchify(iterable, batch_size):
    """Splits an iterable into smaller batches."""
    iterable = iter(iterable)
    while batch := list(islice(iterable, batch_size)):
        yield batch

def save_to_csv(path, prompts, responses, temperature, top_p, top_k):
    """Saves prompts, responses and sampling parameters to a CSV file."""
    with open(path, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for prompt, response in zip(prompts, responses):
            writer.writerow([prompt, response, temperature, top_p, top_k])

def generate_responses(model, prompts, sampling_params):
    """Generate a batch of outputs using vLLM with customizable sampling parameters."""
    outputs = model.chat(prompts, sampling_params=sampling_params, use_tqdm=False)
    
    return [sample.outputs[0].text.replace('"', '') for sample in outputs]

In [44]:
BASE_PROMPT = [{"role": "system", "content": "You are a helpful assistant specializing in writing texts across various domains, including abstracts and news articles, based on provided titles. Based on provided domain and title generate a text of appropriate length related to the domain and title. MAKE SURE TO REPLAY ONLY WITH THE GENERATED TEXT."},
                {"role": "user", "content": "Domain: \n {domain} \n Title: {title}"},
                {"role": "assistant", "content": "Generated text: \n"}]

In [45]:
prompts = [
    [
        BASE_PROMPT[0],  # The system message
        {"role": "user", "content": BASE_PROMPT[1]["content"].format(domain=domain, title=title)},  # Formatted user message
        BASE_PROMPT[2]  # The assistant message
    ]
    for domain, title in df[["domain", "title"]].values
]

KeyError: "None of [Index(['domain', 'title'], dtype='object')] are in the [columns]"

In [46]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

In [47]:
lens = []
batch_size = 128
for prompts_batch in tqdm(batchify(prompts, batch_size), total=len(prompts) // batch_size):
    tokens = tokenizer.apply_chat_template(prompts_batch)
    lens.extend([len(token) for token in tokens])

1254it [00:44, 28.03it/s]                          


In [48]:
too_large = [i for i, l in enumerate(lens) if l > 32_768]
too_large

[]

In [18]:
df.drop(["id", "adv_source_id", "source_id", "model", "decoding", "repetition_penalty", "attack", "domain", "title", "prompt"], axis=1, inplace=True)
df.to_csv("../../data/data_human/raid.csv", index=False)

In [22]:
sampling_params = [
    SamplingParams(temperature=0.0, top_p=1.0, top_k=-1, max_tokens=30_000, seed=SEED),  # Pure Greedy (fully deterministic)
    SamplingParams(temperature=0.2, top_p=1.0, top_k=-1, max_tokens=30_000, seed=SEED),  # Highly Deterministic
    SamplingParams(temperature=0.5, top_p=0.95, top_k=100, max_tokens=30_000, seed=SEED), # Mildly Deterministic but Flexible
    SamplingParams(temperature=0.7, top_p=0.9, top_k=50, max_tokens=30_000, seed=SEED),  # Balanced and Natural
    SamplingParams(temperature=0.9, top_p=0.8, top_k=40, max_tokens=30_000, seed=SEED),  # Slightly More Diverse but Coherent
    SamplingParams(temperature=1.0, top_p=0.95, top_k=30, max_tokens=30_000, seed=SEED), # Default Creative Mode
    SamplingParams(temperature=1.2, top_p=0.7, top_k=20, max_tokens=30_000, seed=SEED),  # Highly Creative
]

In [23]:
llms = ["meta-llama/Llama-3.2-1B-Instruct"]
batch_size = 8
base_path = "../../data/data_ai/raid/raid_"

In [ ]:
for llm in llms:
    model = LLM(model=llm, dtype="half", max_model_len = 10_000)
    csv_path = f"{base_path}{llm.split('/')[-1]}.csv"


    # init csv file
    with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["prompt", "response", "temperature", "top_p", "top_k"])

    cnt = 0
    for prompts_batch in tqdm(batchify(prompts, batch_size), total=len(prompts) // batch_size):
        params = random.choice(sampling_params)
        responses = generate_responses(model, prompts_batch, params)
        save_to_csv(csv_path, prompts_batch, responses, params.temperature, params.top_p, params.top_k)
        cnt += 1
        if cnt > 2:
            break

  0%|          | 2/1253 [05:04<52:56:25, 152.35s/it]


In [50]:
df = pd.read_csv("../../data/data_ai/raid/raid_Llama-3.2-1B-Instruct.csv")
df.head()

,prompt,response,temperature,top_p,top_k
0,"[{'role': 'system', 'content': 'You are a help...",In the rapidly evolving field of artificial in...,1.2,0.7,20
1,"[{'role': 'system', 'content': 'You are a help...",EdgeFlow: A Novel Approach to Practical Intera...,1.2,0.7,20
2,"[{'role': 'system', 'content': 'You are a help...",Semi-supervised contrastive learning has emerg...,1.2,0.7,20
3,"[{'role': 'system', 'content': 'You are a help...",Combining Loss Functions for Improved Multi-Or...,1.2,0.7,20
4,"[{'role': 'system', 'content': 'You are a help...",Attention-Based 3D Seismic Fault Segmentation ...,1.2,0.7,20
